In [1]:
import clip
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, preprocess = clip.load("ViT-B/16")
model.init_adapters()
model = model.to(device)

In [2]:
from RefcocogDataset import RefcocogDataset
from torch.utils.data import DataLoader

train_dataset = RefcocogDataset("./refcocog", split="train", transform=preprocess)
# val_dataset = RefcocogDataset("./refcocog", split="val", transform=preprocess)
# test_dataset = RefcocogDataset("./refcocog", split="test", transform=preprocess)

# print(len(train_dataset), len(val_dataset), len(test_dataset))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)
# val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [3]:
def visualize_sample(sample, bbox, idx=0):
    print(f"Sentence: {sample['sentences'][idx]}")
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(sample['image'][idx].permute(1, 2, 0))
    axes[1].imshow(bbox['gt'][idx])
    plt.tight_layout()
    plt.show()

def build_probability_map(patch_tokens, out_text):
    patch_tokens = patch_tokens.squeeze(0)[1:]
    map = torch.zeros(196)

    for i, token in enumerate(patch_tokens):
        map[i] = 1 - torch.cosine_similarity(token, out_text).item() # 1 - ... temporary fix
    
    map = torch.sigmoid(map)
    map = map.reshape(14, 14)
    return map

def visualize_loss(map, gt, loss_map):
    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    axs[0].imshow(map)
    axs[1].imshow(gt)
    axs[2].imshow(loss_map.reshape(14, 14))
    plt.tight_layout()
    plt.show()

def loss(map, gt, gamma=1):
    map = map.reshape(-1)
    gt = gt.reshape(-1)
    abs = torch.abs(map - gt)
    log = -torch.log(1-abs)
    
    # amplify the error of pixel that should belong to the object
    log = log*(gt*gamma+1)

    loss = torch.sum(log)
    return loss, log

In [4]:
for samples, bbox in train_loader:
    # idx = 7

    # visualize_sample(sample, bbox, idx)

    # image = sample['image'][idx].unsqueeze(0).to(device)
    # sentence = clip.tokenize(sample['sentences'][idx]).to(device)

    images = samples['image'].to(device)
    sentences = clip.tokenize(samples['sentences']).to(device)

    out_image, out_text, patch_tokens, text_tokens = model.encode(images, sentences)
    print(f'Similarity {torch.cosine_similarity(out_image[0].unsqueeze(0), out_text[0].unsqueeze(0)).item():.4f}')
    
    # map = build_probability_map(patch_tokens, out_text)
    # gt = bbox['gt'][idx]/255

    # error, loss_map = loss(map, gt, gamma=3.4)

    # visualize_loss(map, bbox['gt'][idx], loss_map)
    
    break

/Users/francesco/Library/Python/3.9/lib/python/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


Similarity 0.2969
